In [1]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize

from tqdm import tqdm


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aveshverma/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM 

In [ ]:
from datasets import load_dataset, load_from_disk, load_metric